In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import SimpleITK as sitk
import cactas as C
import os
import time
from datetime import datetime

2024-09-28 08:25:54.366704: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
start_time = time.time()
readable_time = datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S.%f')
print(f"Start time: {readable_time}")

Start time: 2024-09-28 08:25:57.493162


In [4]:
def load_seed(seed_image_path, cta_image):
    seed_image = sitk.ReadImage(seed_image_path)
    seed_indices = sitk.GetArrayFromImage(seed_image)
    seeds = list(zip(*seed_indices.nonzero()))
    seeds = [tuple(int(x) for x in seed) for seed in seeds]
    seeds = [(seed[2], seed[1], seed[0]) for seed in seeds]

    if not seeds:
        return None

    last_seeds = [seed[2] for seed in seeds]
    high_range = int(np.median(last_seeds))
    low_range = min(last_seeds)

    if not high_range or not low_range:
        return None

    intensity_values = []
    for seed in seeds:
        # Extract a small region around the seed point
        region_radius = [1, 1, 1]  # Define the size of the region around the seed point
        region = sitk.RegionOfInterest(cta_image, region_radius, seed)
        region_array = sitk.GetArrayFromImage(region)

        # Collect all intensity values of the region
        intensity_values.extend(region_array.flatten())

    if not intensity_values:
        return None

    return seeds, high_range, low_range, intensity_values


In [5]:
DATAPATH='/raid/mpsych/CACTAS/DATA/ESUS/'
TS_CA_PATH='/raid/mpsych/CACTAS/DATA/TS_CA_ESUS'
TS_VERT_PATH='/raid/mpsych/CACTAS/DATA/TS_VERT_ESUS'
output_dir='/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/'

In [6]:
images, labels = C.Helper.load_data(DATAPATH)

In [7]:
len(images), len(labels)

(70, 70)

In [8]:
for image in images:
    num = image.split(".")[0]
    existing_filename = os.path.join(output_dir, num + ".ca.seg.nrrd")
    
    if not os.path.exists(existing_filename):
        # Construct the full path to the CTA image
        cta_image_path = os.path.join(DATAPATH, image)
        cta_image = sitk.ReadImage(cta_image_path)
        cta_image_array = sitk.GetArrayFromImage(cta_image)


        # Assume seed file names are related to the CTA image file name
        seed_image_path_left = os.path.join(TS_CA_PATH + "/" + num, f"common_carotid_artery_left.nii.gz")
        seed_image_path_right = os.path.join(TS_CA_PATH+ "/" + num, f"common_carotid_artery_right.nii.gz")
        left_result = load_seed(seed_image_path_left, cta_image)
        # Load left seeds
        if left_result is not None:
            seeds_left, h_left, l_left, intensities_left = left_result
        else:
            seeds_left, h_left, l_left, intensities_left = None, None, None, None

        # Load right seeds
        right_result = load_seed(seed_image_path_right, cta_image)
        if right_result is not None:
            seeds_right, h_right, l_right, intensities_right = right_result
        else:
            seeds_right, h_right, l_right, intensities_right = None, None, None, None
        
        if not (seeds_left or seeds_right):
            print(f"Skipping {num} due to missing or empty seed data.")
            continue


        # get vertebrae seeds
        seed_image_vt_3 = os.path.join(TS_VERT_PATH + "/" + num, f"vertebrae_C3.nii.gz")
        seed_image_vt_5 = os.path.join(TS_VERT_PATH + "/" + num, f"vertebrae_C5.nii.gz")
        
        # Load high vert seeds
        seeds_high = load_seed(seed_image_vt_3, cta_image)
        if seeds_high is not None:
            seeds_vt_high, h_high, l_high, intensities_vt_high = seeds_high
        else:
            seeds_vt_high, h_high, l_high, intensities_vt_high = None, None, None, None
        
        # Load low vert seeds
        seeds_low = load_seed(seed_image_vt_5, cta_image)
        if seeds_low is not None:
            seeds_vt_low, h_low, l_low, intensities_vt_low = seeds_low
        else:
            seeds_vt_low, h_low, l_low, intensities_vt_low = None, None, None, None

        # Combine seeds and intensity values from both sides
        seeds = (seeds_left if seeds_left is not None else []) + (seeds_right if seeds_right is not None else [])
        intensity_values = ((intensities_left if intensities_left is not None else []) +
                    (intensities_right if intensities_right is not None else []))

        intensity_values_array = np.array(intensity_values)  
        mean_intensity = np.mean(intensity_values_array)

        lower_threshold = float(mean_intensity) - 50
        upper_threshold = lower_threshold + 200
        
        
        # Based on seeds, set the range
        if (h_left is not None and h_left < l_low) or (h_right is not None and h_right < l_low):
            # Calculate min_range, ignoring None values
            min_range = min(h for h in [h_left, h_right] if h is not None)
            cut_z_range=[min_range, h_high]
        else:
            cut_z_range = [l_low, h_high]
        
        
        # cut image based on vertebrae seeds
        cta_image_array_new = np.zeros_like(cta_image_array)
        cta_image_array_new[cut_z_range[0]:cut_z_range[1]] = cta_image_array[cut_z_range[0]:cut_z_range[1]]

        # Add header and make output file c
        cta_image_modified = sitk.GetImageFromArray(cta_image_array_new)
        cta_image_modified.CopyInformation(cta_image)
        cta_image_modified.SetSpacing(cta_image.GetSpacing())
        cta_image_modified.SetOrigin(cta_image.GetOrigin())
        cta_image_modified.SetDirection(cta_image.GetDirection())


        # Use adjusted thresholds in the ConnectedThreshold function
        output_image = sitk.ConnectedThreshold(cta_image_modified, seedList=seeds,
                                               lower=lower_threshold, upper=upper_threshold)

        # Process the output image as before
        output_image_array = sitk.GetArrayFromImage(output_image)
        output_image_array[:l_low] = np.zeros_like(output_image_array[:l_low])

        output_image_modified = sitk.GetImageFromArray(output_image_array)
        output_image_modified.CopyInformation(output_image)
        output_image_modified.SetSpacing(output_image.GetSpacing())
        output_image_modified.SetOrigin(output_image.GetOrigin())
        output_image_modified.SetDirection(output_image.GetDirection())

        # Save the output image with a unique name
        output_image_path = os.path.join(output_dir, f"{num}.ca.seg.nrrd")
        print(output_image_path)
        sitk.WriteImage(output_image_modified, output_image_path)

        #print(str(num) + " " +str(lower_threshold) + " " + str(upper_threshold))
    else:
        print(f"{existing_filename} already exists. Skipping.")

/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/5.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/18.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/24.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/60.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/95.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/9.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/23.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/97.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/55.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/69.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/45.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/35.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/21.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/51.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/66.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/91.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/63.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/42.ca.seg.nrrd
/raid/mpsych/CACTAS/DATA/CA_ESUS35_v2/15.ca.seg.

In [9]:
end_time = time.time()
readable_time = datetime.fromtimestamp(end_time).strftime('%Y-%m-%d %H:%M:%S.%f')
print(f"End time: {readable_time}")

End time: 2024-09-28 13:03:08.614025


In [10]:
execution_time = end_time - start_time
readable_time = datetime.fromtimestamp(execution_time).strftime('%Y-%m-%d %H:%M:%S.%f')
print(f"Execution time: {readable_time}")

Execution time: 1969-12-31 23:37:11.120862
